In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig_dir = "figures"
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
from avapi.carla import CarlaScenesManager

data_root = "/data/shared/CARLA/multi-agent-intersection/"
CSM = CarlaScenesManager(data_dir=data_root, split="train")
idx_scene = 0
CDM = CSM.get_scene_dataset_by_index(idx_scene)

## Make FOV Estimator Figures

In [ ]:
from avstack.modules.perception.fov_estimator import (
    FastRayTraceBevLidarFovEstimator,
    SlowRayTraceBevLidarFovEstimator,
    ConcaveHullLidarFOVEstimator,
)

# set up fov models
fov_estimator_hull = ConcaveHullLidarFOVEstimator()
fov_estimator_fast = FastRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)
fov_estimator_slow = SlowRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)

# get some random lidar point clouds to analyze
idx_frame = 120
frames = CDM.get_frames(sensor="lidar-0", agent=0)
objs_view = CDM.get_objects_global(frame=frames[idx_frame], ignore_static_agents=True)
# objs_view = CDM.get_objects(frame=frames[idx_frame], sensor="lidar-0", agent=0, whitelist_types="all")
pc = CDM.get_lidar(frame=frames[idx_frame], sensor="lidar-0", agent=0)
img_front = CDM.get_image(frame=frames[idx_frame], sensor="camera-0", agent=0)
img_back = CDM.get_image(frame=frames[idx_frame], sensor="camera-1", agent=0)

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes, show_lidar_bev_with_boxes
from PIL import Image

fov_out_dir = os.path.join(fig_dir, "field_of_view")
os.makedirs(fov_out_dir, exist_ok=True)

# run fov estimator on this frame
fov_hull = pc.concave_hull_bev(concavity=1)
fov_fast = fov_estimator_fast(pc)
fov_slow = fov_estimator_slow(pc)

# project lidar to bev for plotting
pc_bev = pc.project_to_2d_bev()

# filter the objects to show
agent = CDM.get_agent(frame=frames[idx_frame], agent=0)
objs_downselect = [obj for obj in objs_view if 4 < obj.distance(agent.position) < 10]
box_colors = ["lightblue", "brown"]
img_front_view = show_image_with_boxes(
    img=img_front,
    boxes=objs_downselect,
    box_colors=box_colors,
    box_thickness=5,
    inline=True,
    return_image=True,
)
im = Image.fromarray(img_front_view)
im.save(os.path.join(fov_out_dir, "front_view_image.pdf"))

# make plot of lidar data in BEV
models = [fov_fast, fov_slow, fov_hull]
labels = ["Quantized Ray Trace", "Continuous Ray Trace", "Concave Hull"]
filenames = ["quantized.pdf", "continuous.pdf", "hull.pdf"]
colors = ["#C79FEF", "#13EAC9", "#069Af3"]
point_color = "black"
for model, label, color, filename in zip(models, labels, colors, filenames):
    img_fov_model = show_lidar_bev_with_boxes(
        pc=pc,
        boxes=objs_downselect,
        background_color="white",
        colormethod="black",
        box_colors=box_colors,
        box_thickness=10,
        fov=model,
        fov_color=color,
        fov_filled=True,
        fov_filled_alpha=0.5,
        bev_size=(600, 600),
        inline=True,
        flipx=False,
        flipxy=False,
        return_image=True,
    )
    im = Image.fromarray(img_fov_model)
    im.save(os.path.join(fov_out_dir, filename))

In [ ]:
# fig, axs = plt.subplots(1, 3, figsize=(12,6))

# # fill the polygon path
# polygon = MplPolygon(model.boundary, closed=True, facecolor=color, alpha=0.3)
# ax.add_patch(polygon)

# # show the fov border
# ax.plot(model.boundary[:,0], model.boundary[:,1], linestyle="-", color=color)

# # show the point cloud
# ax.plot(pc_bev.data.x[:,0], pc_bev.data.x[:,1], color=point_color,
#         marker=".", linestyle="", alpha=0.2, markersize=2, label="Point Cloud")

# # set figure things
# ax.set_title(f"{label} Model")
# ax.set_aspect('equal', adjustable='box')
# ax.set_axis_off()
# plt.show()

# show the image
# show_image_with_boxes(img_back, boxes=objs_downselect, box_colors=box_colors, inline=True)

## Run a simple unit test on the fov estimator

In [ ]:
from avstack.geometry import Position, ReferenceFrame, GlobalOrigin3D


def make_far_reference():
    return ReferenceFrame(
        x=np.array([100, 100, 0]), q=np.quaternion(1), reference=GlobalOrigin3D
    )


pc_local = CDM.get_lidar(frame=frames[0], sensor="lidar-0", agent=0)

# run fov estimator in a simple frame
fov_estimator = FastRayTraceBevLidarFovEstimator()
fov_local = fov_estimator(pc_local, in_global=False)
pt_local = Position(np.array([-20, 0, 0], dtype=float), reference=pc_local.reference)
assert fov_local.check_point(pt_local.x)
10
# convert the fov frame using change reference
pc_far = pc_local.change_calibration(make_far_reference(), inplace=False)
pt_far = pt_local.change_reference(pc_far.reference, inplace=False)
fov_local_as_far = fov_local.change_reference(pc_far.reference, inplace=False)
assert fov_local_as_far.check_point(pt_far.x)

# run fov estimator in a different frame
fov_far = fov_estimator(pc_far, in_global=False)
assert fov_far.check_point(pt_far.x)

In [ ]:
# show in a local frame
plt.plot(
    pc_local.data.x[:, 0],
    pc_local.data.x[:, 1],
    "bo",
    alpha=0.2,
    markersize=2,
    label="Point Cloud",
)
plt.plot(
    fov_local.boundary[:, 0], fov_local.boundary[:, 1], "k--", label="Fast FOV Model"
)
plt.plot(pt_local.x[0], pt_local.x[1], "gx", markersize=10, label="point")
plt.show()

# show in the transformed frame
plt.plot(
    pc_far.data.x[:, 0],
    pc_far.data.x[:, 1],
    "bo",
    alpha=0.2,
    markersize=2,
    label="Point Cloud",
)
plt.plot(
    fov_local_as_far.boundary[:, 0],
    fov_local_as_far.boundary[:, 1],
    "k--",
    label="Fast FOV Model",
)
plt.plot(pt_far.x[0], pt_far.x[1], "gx", markersize=10, label="point")
plt.show()

# show in the far frame
plt.plot(
    pc_far.data.x[:, 0],
    pc_far.data.x[:, 1],
    "bo",
    alpha=0.2,
    markersize=2,
    label="Point Cloud",
)
plt.plot(fov_far.boundary[:, 0], fov_far.boundary[:, 1], "k--", label="Fast FOV Model")
plt.plot(pt_far.x[0], pt_far.x[1], "gx", markersize=10, label="point")
plt.show()